# Intro to Data Science @ SzISz Part III.
## Dimensionality Reduction

### Table of contents
- <a href="#What-is-Dimensionality-Reduction?">Theory</a>
- <a href="#Feature-Selection">Feature Selection</a>
- <a href="#Matrix-Decomposition">Matrix Decomposition</a>
- <a href="#Other-Techniques">Other Techniques</a>

## What is Dimensionality Reduction?
Dimensionality reduction _"is the process of reducing the number of random variables under consideration, and can be divided into feature selection and feature extraction."_

_"__Feature selection__ approaches try to find a subset of the original variables. ... In some cases, data analysis such as regression or classification can be done in the reduced space more accurately than in the original space."_

_"__Feature extraction__ transforms the data in the high-dimensional space to a space of fewer dimensions. The data transformation may be linear, as in principal component analysis (PCA), but many nonlinear dimensionality reduction techniques also exist."_ from:<a href="https://en.wikipedia.org/wiki/Dimensionality_reduction">Wikipedia</a>


## Why is it important?
With hundreds of features in the datasets, there will always be some which does not contribute to the overall precision of the predictive model. These features could be redundant, overlapping or linear combination of each other or simply irrelevant to the prediction. To improve training and transformation/prediction time, it is crucial to reduce the number of features to a moderate amount.

The <a href="https://en.wikipedia.org/wiki/Curse_of_dimensionality">curse of dimensionality</a> also requires one to deal with the dimensionality concerns.


## Tools
- <a href="http://scikit-learn.org/stable/modules/feature_selection.html">Feature Selection</a>
- <a href="http://scikit-learn.org/stable/modules/decomposition.html#decompositions">Matrix decomposition</a>
- <a href="http://scikit-learn.org/stable/modules/feature_extraction.html#feature-hashing">Hashing</a>
- etc.

In [1]:
%matplotlib inline
import numpy as np
import scipy.sparse as sp
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_iris

In [2]:
iris = load_iris()
X, y = iris.data, iris.target

## Feature Selection

In [3]:
#minél több dimenziónk van, annál több adat kell egy item leírásához
#egyre több futásidő, egyre többmemória
#vagy manuálisan vagy vmelyien módszerrel kiválasztjuka számunkra leginkább információt adó feature-et
#variance treshold: bizonyos tresholdnál kisebb varianciájú változókat elhagyunk
from sklearn.svm import SVR, LinearSVC
from sklearn.feature_selection import VarianceThreshold, RFE, SelectFromModel

__VarianceThreshold__:

In [5]:
#nem veszi figyelembe, ha a változók különböző skálán vannak
#érdemes először standardizálni
thres=VarianceThreshold(0.4)
X_t=thres.fit_transform(X)
X_t.shape()

TypeError: 'tuple' object is not callable

__RFE__:

In [10]:
#rekursive feature elimination
#osztályozó: először benne van az összes feature, majd kivesz belőle és emgnézi romlik-e vele az eredmény
#külső becslő kell neki
#support vector machines: próbálunkolyan egyenest húzni, ami az osztályokat elválasztja egymástól
#cél a margómaximalizálása
#kernel függvények: adathalmaz transzformációja, ahol már lineárisan szeparálható a feladat
#nem kell minden tagok áttranszformálni az új térbe, elég bővíteni a távolságdefiníciót
svr=SVR(kernel="linear", C=0.01)
rfe=RFE(svr, n_features_to_select=2)
X_t= rfe.fit_transform(X,y)

In [9]:
rfe.ranking_ #a paraméterek értékei, meddig maradtak bent,mikor estek ki
rfe.support_#mit hagyjunk ki, mit hagyjunk bent

array([False, False,  True,  True], dtype=bool)

__Select from model__:

In [13]:
svc=LinearSVC(C=0.01).fit(X, y)
sel=SelectFromModel(svc, prefit=True) #it is fotosság szerint paramétereket számít, threshold: benthagyni vs. kivenni
#egyébként pedig azt hagyja bent, ami jobb az átlagnál
X_t=sel.transform(X)
X_t.shape

(150L, 2L)

## Matrix Decomposition

In [6]:
from sklearn.datasets import fetch_20newsgroups_vectorized
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA, TruncatedSVD

In [14]:
#már eleve a tfidf-el transzformált anyagot töltjük le
data2 = fetch_20newsgroups_vectorized(subset='test', remove=('headers', 'footers', 'quotes'))
X2, y2 = data2.data, data2.target

__PCA__:

In [ ]:
#PCA: 
pca=PCA('mle')
X2_t=pca.fit_transform(X2.todense())
X2_t.shape

__SVD__:

In [15]:
svd=TruncatedSVD(n_components=300)
X2_t=svd.fit_transform(X2)
X2_t.shape

(7532L, 300L)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
baseline=mean_squared_error(y2, np.ones(y2.shape)*y2.mean())
knn=KNeighborsClassifier()
knn.fit(X2_t, y2)
mean_squared_error(y2, knn.predict(X2_t))

## Other Techniques

In [2]:
#minden szónak vezsi a hash-ét
#
from sklearn.feature_extraction.text import HashingVectorizer, TfidfTransformer
vect=HashingVectorizer(n_features=1024) #1024 értékre fog leképződni az eredetileg 30 akárhányezer szó


In [3]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA, TruncatedSVD
data3= fetch_20newsgroups()
X3, y3 = data3.data, data3.target
vect.fit(X3, y3)

HashingVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
         dtype=<type 'numpy.float64'>, encoding=u'utf-8', input=u'content',
         lowercase=True, n_features=1048576, ngram_range=(1, 1),
         non_negative=False, norm=u'l2', preprocessor=None,
         stop_words=None, strip_accents=None,
         token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None)

In [8]:
X3[0]
vect.transform([X3[0]])
vect=HashingVectorizer(n_features=256)
#bemenetként dokumentumok listáját várja
X3_t=vect.transform(X3).todense()

In [9]:
tfidf=TfidfTransformer()
X3_tt=tfidf.fit_transform(X3_t)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
knn=KNeighborsClassifier()
knn.fit(X3_tt, y3)
knn.predict(X3_tt)
sns.heatmap(confusion_matrix(y3,(knn.predict(X3_tt))))
#sorok: valós, oszlopok:prediktált, ha a főátlóban sötétebb, akkor jó a besorolás


In [ ]:
#adott modellpontossága:
knn.score(X3_tt, y3)

In [ ]:
#rendezzük az egészet egy pipeline-ba
#kimásolni!!!!
pipe=Pipiline([('hash', HashingVectorizer, n_features=256), ('tfidf')])

pipe.score(X3, y3)
sns.heatmap(confusion_matrix(y3, pipe.predict(X3)))

